In [307]:
# Load the necessary libraries
import numpy as np
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# This is for pretty printing
import IPython.display as disp

Constants

In [308]:
m1, m2, m3 = 1, 2, 0.5
L1, L2 = 0.5, 1.5
I1, I2 = 0.5, 1.5
F0 = 20
k = 1
g = 9.81
n = 4
theta_crank_0 = np.pi/2
phi = [2 * sp.pi * i / n for i in range(n)]
initial_angle = [float(theta_crank_0 + phi[i].evalf()) for i in range(n)]
dtheta1 = 10



In [309]:
t = sp.symbols('t')

x1, y1, theta1, x2, y2, theta2, x3, y3, tau = [], [], [], [], [], [], [], [], []
q, dq, x_com_1, x_com_2, x_com_3 = [], [], [], [], []

# Create dynamic symbol arrays
for i in range(n):
    x1.append(dynamicsymbols(f'x1{i}'))
    y1.append(dynamicsymbols(f'y1{i}'))
    theta1.append(dynamicsymbols(f'theta1{i}'))
    
    x2.append(dynamicsymbols(f'x2{i}'))
    y2.append(dynamicsymbols(f'y2{i}'))
    theta2.append(dynamicsymbols(f'theta2{i}'))
    
    x3.append(dynamicsymbols(f'x3{i}'))
    y3.append(dynamicsymbols(f'y3{i}'))    
    

    q.append(sp.Matrix([x1[i], y1[i], theta1[i], x2[i], y2[i], theta2[i], x3[i], y3[i]]))
    
    
    # COM vectors
    x_com_1.append(sp.Matrix([x1[i], y1[i]]))
    x_com_2.append(sp.Matrix([x2[i], y2[i]]))
    x_com_3.append(sp.Matrix([x3[i], y3[i]]))

R = lambda theta: sp.Matrix([[sp.cos(theta), -sp.sin(theta)], [sp.sin(theta), sp.cos(theta)]])

M = np.diag([m1, m1, I1, m2, m2, I2, m3, m3])
W = np.linalg.inv(M)

Q = []
for i in range(n):
    Q.append(sp.Matrix([0, -m1*g, -k*theta1[i].diff(t), 0, -m2*g, 0, 0, -m3*g + F0 * sp.cos(theta1[i])]))


In [310]:
i_cap = sp.Matrix([1, 0])
j_cap = sp.Matrix([0, 1])

C1, C2, C3, C4, C5, C6, C7, C = [], [], [], [], [], [], [], []

for i in range(n):
    constraint_1 = x_com_1[i] + R(theta1[i]) @ sp.Matrix([-L1/2, 0])
    C1.append(constraint_1.dot(i_cap))
    C2.append(constraint_1.dot(j_cap))


    constraint_2 = x_com_1[i] - x_com_2[i] + R(theta1[i]) @ sp.Matrix([L1/2, 0]) - R(theta2[i]) @ sp.Matrix([-L2/2, 0])
    C3.append(constraint_2.dot(i_cap))
    C4.append(constraint_2.dot(j_cap))
    

    constraint_3 = x_com_2[i] + R(theta2[i]) @ sp.Matrix([L2/2, 0]) - x_com_3[i]
    C5.append(constraint_3.dot(i_cap))
    C6.append(constraint_3.dot(j_cap))


    constraint_4 = x_com_3[i][0]
    C7.append(constraint_4)

 
    C.append(sp.Matrix([C1[i], C2[i], C3[i], C4[i], C5[i], C6[i], C7[i]]))


In [311]:
J, dq, dC, dJ, JWJT, RHS = [], [], [], [], [], []
JWJT_fn, RHS_fn, C_fn, J_fn, dC_fn, dJ_fn, Q_fn = [], [], [], [], [], [], []

for i in range(n):
    J.append(C[i].jacobian(q[i]))
    dq.append(q[i].diff(t))
    dC.append(J[i] @ dq[i])
    dJ.append(dC[i].jacobian(q[i]))
    JWJT.append(J[i] @ W @ J[i].T)
    RHS.append(-dJ[i] @ dq[i] - J[i] @ W @ Q[i] - 1 * C[i] - 1 * dC[i])
    
    JWJT_fn.append(sp.lambdify(args=(q[i], dq[i]), expr=JWJT[i]))
    RHS_fn.append(sp.lambdify(args=(q[i], dq[i]), expr=RHS[i]))
    C_fn.append(sp.lambdify(args=(q[i], dq[i]), expr=C[i]))   
    J_fn.append(sp.lambdify(args=(q[i], dq[i]), expr=J[i])) 
    dC_fn.append(sp.lambdify(args=(q[i], dq[i]), expr=dC[i]))
    dJ_fn.append(sp.lambdify(args=(q[i], dq[i]), expr=dJ[i]))
    Q_fn.append(sp.lambdify(args=(q[i], dq[i]), expr=Q[i]))

In [312]:
initial_position_body_1, initial_position_body_2, initial_position_body_3 = [], [], []
initial_velocity_body_1, initial_velocity_body_2, initial_velocity_body_3 = [], [], []
x0 = np.empty((n, 16))

def compute_initial_crank_position(L1, initial_angle):
    return R(initial_angle) @ sp.Matrix([L1/2, 0])

def compute_initial_rod2_position(L1, L2, initial_angle):
    # Position of the crank pin (end of crank arm)
    crank_pin = R(initial_angle) @ sp.Matrix([L1, 0])
    
    # y-coordinate of piston pin must satisfy: distance = L2 (rod length)
    # We assume piston is vertically above or below the crank pin, so x = 0
    y_piston = crank_pin[1]
    y_offset = sp.sqrt(sp.Max(0, L2**2 - crank_pin[0]**2))
    piston_pin = sp.Matrix([0, crank_pin[1] + y_offset])
    
    # Rod center = midpoint of crank pin and piston pin
    return (crank_pin + piston_pin) / 2

def compute_initial_piston_position(L1, L2, initial_angle):
    crank_pin = R(initial_angle) @ sp.Matrix([L1, 0])
    y_offset = sp.sqrt(sp.Max(0, L2**2 - crank_pin[0]**2))

    return sp.Matrix([0, crank_pin[1] + y_offset])  # piston COM offset vertically

def compute_rod2_angle(L1, initial_angle, y_piston):
    crank_pin = R(initial_angle) @ sp.Matrix([L1, 0])
    piston_pos = sp.Matrix([0, y_piston])
    vec = piston_pos - crank_pin
    return sp.atan2(vec[1], vec[0])

for i in range(n):
    xy_body1 = compute_initial_crank_position(L1, initial_angle[i]).evalf()
    xy_body2 = compute_initial_rod2_position(L1, L2, initial_angle[i]).evalf()
    xy_body3 = compute_initial_piston_position(L1, L2, initial_angle[i]).evalf()
    theta_body2 = float(compute_rod2_angle(L1, initial_angle[i], xy_body3[1]).evalf())


    initial_position_body_1.append(np.array([float(xy_body1[0]), float(xy_body1[1]), initial_angle[i]]))
    initial_position_body_2.append(np.array([float(xy_body2[0]), float(xy_body2[1]), theta_body2]))
    initial_position_body_3.append(np.array([float(xy_body3[0]), float(xy_body3[1])]))
    initial_velocity_body_1.append(np.array([0, 0, dtheta1])) # To start the engine
    initial_velocity_body_2.append(np.array([0, 0, 0]))
    initial_velocity_body_3.append(np.array([0, 0]))
    x0[i] = np.concatenate((initial_position_body_1[i], initial_position_body_2[i], initial_position_body_3[i],
        initial_velocity_body_1[i], initial_velocity_body_2[i], initial_velocity_body_3[i]))



x0_first_half = np.empty((n, 8))
x0_second_half = np.empty((n, 8))


for i in range(n):
    x0_first_half[i] = x0[i][:8]
    x0_second_half[i] = x0[i][8:]




Calculate initial conditions for the system

In [313]:
import scipy.optimize as opt

x = []
initial_guess, b, dx, C_val, dC_val = [], [], [], [], []
x0 = np.empty((n, 16))

for i in range(n):
    x.append(x0_first_half[i])
    
    def optimiser(b_i):
        dx1, dy1, dx2, dy2, dtheta2, dx3, dy3 = b_i
        dq_i = np.array([dx1, dy1, dtheta1, dx2, dy2, dtheta2, dx3, dy3])
        val = dC_fn[i](x[i], dq_i).flatten()
        return val
        
    initial_guess.append(np.array([0, 0, 0, 0, 0, 0, 0]))
    result = opt.root(optimiser, initial_guess[i])
    print(result)

    b.append(result.x)
    dx.append(np.array([b[i][0], b[i][1], dtheta1, b[i][2], b[i][3], b[i][4], b[i][5], b[i][6]]))
        
    C_val.append(C_fn[i](x[i], dx[i]))
    dC_val.append(dC_fn[i](x[i], dx[i]))
        
    print(f'Position constraint: {C_val[i]}')
    print(f'Velocity constraint: {dC_val[i]}')


    print("C_val[i] =", C_val[i])
    print("dx[i] =", dx[i])
    print("x[i] =", x[i])
    assert np.allclose(C_val[i], 0), "Initial position constraint violated"
    assert np.allclose(dC_val[i], 0), "Initial velocity constraint violated"
    x0[i] = np.concatenate((x[i], dx[i]))
print(x0)

 message: The solution converged.
 success: True
  status: 1
     fun: [ 0.000e+00  0.000e+00  0.000e+00  2.465e-32  0.000e+00
            0.000e+00  0.000e+00]
       x: [-2.500e+00  1.531e-16 -2.500e+00  7.082e-16 -3.333e+00
            0.000e+00  1.110e-15]
  method: hybr
    nfev: 12
    fjac: [[-7.071e-01 -3.081e-33 ...  3.081e-33  0.000e+00]
           [-5.170e-26 -7.071e-01 ...  0.000e+00  0.000e+00]
           ...
           [-4.800e-42  2.985e-26 ... -2.985e-26 -1.000e+00]
           [ 9.285e-17 -5.774e-01 ...  5.774e-01 -5.170e-26]]
       r: [-1.414e+00  5.170e-26 ...  2.985e-26 -5.774e-01]
     qtf: [-6.280e-16 -8.239e-25  3.626e-16 -4.757e-25 -1.026e-15
           -3.478e-50  6.727e-25]
Position constraint: [[ 0.00000000e+00]
 [ 0.00000000e+00]
 [-1.05301114e-16]
 [ 0.00000000e+00]
 [-1.05301114e-16]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]
Velocity constraint: [[0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [2.46519033e-32]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.000

In [314]:
def piston_engine(t, state):
    '''
    This function returns the derivative of the state vector for the system

    Parameters:
    t: float
        The current time
    state: numpy array
        The current state of the system
        The vector is arranged as [q, dq]
        where q is the position vector and dq is the derivative of the position vector
    '''

    q, dq = np.split(state, 2)

    # Solve for lambda 
    lam = np.linalg.solve(JWJT_fn[i](q,dq), RHS_fn[i](q,dq))

    # Solve for constraint forces 
    Qhat = J_fn[i](q, dq).T @ lam

    # Calculate accelerations
    ddq = W @ (Q_fn[i](q, dq) + Qhat)
    ddq = ddq.flatten()

    return np.concatenate((dq, ddq))

# Test run
for i in range(n):
    piston_engine(0, x0[i])

In [ ]:
time_points = 500
t_span = (0, 30)
t_eval = np.linspace(*t_span, time_points)
sol = []
for i in range(n):
    sol.append(solve_ivp(piston_engine, t_span, x0[i], atol=1e-7, rtol=1e-7, method='BDF', t_eval=t_eval))

Animation

In [316]:
# Class for drawing the box
class Box:
    def __init__(self, width, height, color='b'):
        self.width = width
        self.height = height
        self.color = color
        self.offset = -np.array([width/2, height/2])

    def first_draw(self, ax):
        corner = np.array([0, 0])
        self.patch = plt.Rectangle(corner, 0, 0, angle=0, 
                        rotation_point='center', color=self.color, animated=True)
        ax.add_patch(self.patch)
        self.ax = ax
        return self.patch
    
    def set_data(self, x, y, theta):
        self.x = x
        self.y = y
        self.theta = theta

    def update(self, i):
        x, y, theta = self.x[i], self.y[i], self.theta[i]
        theta = np.rad2deg(theta)

        # The rectangle is drawn from the left bottom corner
        # So, we need to calculate the corner position
        corner = np.array([x, y]) + self.offset

        # Update the values for the rectangle
        self.patch.set_width(self.width)
        self.patch.set_height(self.height)
        self.patch.set_xy(corner)
        self.patch.set_angle(theta)
        return self.patch

In [317]:
### NOTE: This might take a while to run.

from matplotlib.animation import FuncAnimation
from IPython.display import HTML

fig, ax = plt.subplots()
# Close the figure as we will be making an animation
plt.close()

# Set bounding limits 
ax.set_ylim(-0.6, 2.1)
ax.set_xlim(-0.6, 0.6)
ax.set_aspect('equal')

# Get the position and angle of the two bodies
x1, y1, theta1, x2, y2, theta2, x3, y3, theta3 = (
    np.empty((n, sol[0].y[0].shape[0])),
    np.empty((n, sol[0].y[1].shape[0])),
    np.empty((n, sol[0].y[2].shape[0])),
    np.empty((n, sol[0].y[3].shape[0])),
    np.empty((n, sol[0].y[4].shape[0])),
    np.empty((n, sol[0].y[5].shape[0])),
    np.empty((n, sol[0].y[6].shape[0])),
    np.empty((n, sol[0].y[7].shape[0])),
    np.empty((n, sol[0].y[8].shape[0])),
)

for i in range(n):
    print(f"\n\n{i}\n\n")
    x1[i], y1[i], theta1[i] = sol[i].y[:3]
    x2[i], y2[i], theta2[i] = sol[i].y[3:6]
    x3[i], y3[i] = sol[i].y[6:8]
    theta3[i] = np.zeros_like(x3[i])

box1_list = [Box(L1, 0.01, 'b') for _ in range(n)]
box2_list = [Box(L2, 0.01, 'r') for _ in range(n)]
box3_list = [Box(0.1, 0.3, 'g') for _ in range(n)]

for i in range(n):
    box1_list[i].set_data(x1[i], y1[i], theta1[i])
    box2_list[i].set_data(x2[i], y2[i], theta2[i])
    box3_list[i].set_data(x3[i], y3[i], theta3[i])


boxes = box1_list + box3_list + box2_list

def init():
    ax.set_title("t=0.00 sec", fontsize=15)
    for box in boxes:
        box.first_draw(ax)
    patches = [box.patch for box in boxes]
    return patches

def animate(i):
    ''' Draw the i-th frame of the animation'''
    
    ax.set_title(f"t={sol[0].t[i]:.2f} sec", fontsize=15)
    for box in boxes:
        box.update(i)
    patches = [box.patch for box in boxes]
    return patches

# Set the interval between frames
dt = sol[0].t[1] - sol[0].t[0]

# Create the animation
anim = FuncAnimation(fig, animate, frames=len(sol[0].t), init_func=init, blit=False, interval=1000*dt)
HTML(anim.to_html5_video())



0




1




2




3


